In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.optim as optim

In [12]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# Load pretrained VGG16 without classifier
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.features.parameters():
    param.requires_grad = False  # Freeze convolutional layers

C:\Users\singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\singh/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [01:23<00:00, 6.67MB/s] 


In [4]:
# Modify classifier for binary classification
vgg16.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(25088, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
)
vgg16 = vgg16.to(device)

In [5]:
# Data transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor()
])

In [6]:
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [8]:
# Dataset and loaders
train_dataset = datasets.ImageFolder('../cat_dog/train', transform=train_transforms)
val_dataset = datasets.ImageFolder('../cat_dog/validation', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [9]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(vgg16.parameters(), lr=0.0001)

In [11]:
print(torch.cuda.is_available())  # Check weather graphic card is available
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3060 Laptop GPU


In [13]:
# Training loop
for epoch in range(10):
    vgg16.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

C:\Users\singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 0.24034400282513546
Epoch 2, Loss: 0.22258772015192552
Epoch 3, Loss: 0.21777279624072848
Epoch 4, Loss: 0.21392548667631203
Epoch 5, Loss: 0.20404779597737832
Epoch 6, Loss: 0.19957695753329888
Epoch 7, Loss: 0.2018310140125418
Epoch 8, Loss: 0.19750438817437674
Epoch 9, Loss: 0.19238730406296842
Epoch 10, Loss: 0.18882246792245366


In [14]:
# Save the model's state_dict
torch.save(vgg16.state_dict(), 'vgg16_cat_dog.pth')
print("Model saved as vgg16_cat_dog.pth")


Model saved as vgg16_cat_dog.pth
